Book Recommendation Practice

In [ ]:
#basic Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
#importing dataset

books=pd.read_csv('./Books.csv')
user=pd.read_csv('./Users.csv')
ratings=pd.read_csv('./Ratings.csv')

In [ ]:
books.head(4)
# books.isna().sum()

In [ ]:
user.head(4)
# user.isna().sum()

In [ ]:
ratings.head(4)
# ratings.isna().sum()

In [ ]:
# merge book in rating with ISBN
ratings_with_name = ratings.merge(books, on='ISBN')
ratings_with_name.head()

In [ ]:
# counting rating for user-id and >50
user_rating_count=ratings_with_name.groupby('User-ID').count()['Book-Rating']
active_users=user_rating_count[user_rating_count>50].index
# filtering rating for active user(>50)
filtered_ratings=ratings_with_name[ratings_with_name['User-ID'].isin(active_users)]
#combining rating for Book-Title and popular if>=10
book_ratings_count=filtered_ratings.groupby('Book-Title').count()['Book-Rating']
popular_books=book_ratings_count[book_ratings_count>=10].index
# filtering filtered_rating  for Book-Title
final_rating=filtered_ratings[filtered_ratings['Book-Title'].isin(popular_books)]
final_rating.head()

We need to work with rating book-title and user-id

In [ ]:
new_table=final_rating.pivot_table(index='Book-Title',columns='User-ID',values='Book-Rating')
new_table.fillna(0, inplace=True)
new_table.head()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
similarity=cosine_similarity(new_table)

In [ ]:
def recommend_book(book_name):
    # get the index for given book
    try:
        index=np.where(new_table.index==book_name)[0][0]
        similar_books=sorted(list(enumerate(similarity[index])),key=lambda x: x[1], reverse=True)[1:6]
        print(f"Books similar to {book_name}:")
        for i in similar_books:
            book_title = new_table.index[i[0]]
            print(book_title)
    except:
        print('Book - ',book_name,' Not found in Database')

In [ ]:
recommend_book('Far from the MaddingCrowd')

Till now recommending book on the basis of similarity matrix...

In [ ]:
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix

A sparse matrix is a matrix in which most of the elements are zero. Sparse matrices are used to efficiently store and manipulate large datasets where the majority of elements are zeros.

In [ ]:
# converting new_table (pivot table ) into sparse matrice
nt_matrix = csr_matrix(new_table.values)
print(nt_matrix.toarray())
# Convert the pivot table into a sparse matrix
model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
model_knn.fit(nt_matrix)

In [ ]:
def recommend(book):
    try:
        query_index = new_table.index.get_loc(book)
        distances, indices = model_knn.kneighbors(nt_matrix[query_index], n_neighbors=6)

        print(f"Recommendations for '{new_table.index[query_index]}':")
        for i in range(1, len(distances.flatten())):
            print(f"{i}: {new_table.index[indices.flatten()[i]]}")

    except KeyError:
        print(f"The book '{book}' is not found in the dataset.")
        similar_books = find_similar_books(book)

        if similar_books:
            print("*********************************************")
            print(f"Did you mean one of these books?")
            for i, similar_book in enumerate(similar_books, start=1):
                print(f"{i}: {similar_book}")
        else:
            print(f"No similar books found. Please try a different book name (Type 'exit' to stop).")



def find_similar_books(book):
    similar_books = [b for b in new_table.index if book.lower() in b.lower()]
    return similar_books[:10] if similar_books else []


while True:
    print("*********************************************")
    book_name = input("Enter the Book Name (Type 'exit' to stop): ")

    if book_name.lower() == 'exit':
        print("Exiting the recommendation system.")
        print("*********************************************")
        break

    recommend(book_name)
